Evalvuate the eror 
The objective of the eval pred is to first figure out some info about the evaluation metric. The evaluation metric Consists of two parts. Ftist is a rank based one. We rank the customers that have defulted based on the predicted probability. choose the highest .04(#1 + #0*20). 

This is equvalent to the recall level of 4% the doc states. That is, of all the customers that are going to deafult, we aim to recall at least 96% of them!! 


# Q:
 - What does a recall of 4% mean? How do we train for a recall of a 4%? 

In [3]:
import torch 
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from pd.nn.conv import Conv
from pd.data.loader import CustomerData
from pd.nn.train_utils import train_torch_model
from pd.metric import amex_metric
from pd.params import *


In [ ]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from bes.nn.es_module import ESModule


In [7]:
def top_four_percent_captured(df) -> float:
        """Corresponds to the recall for a threshold of 4 %"""
        df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
        four_pct_cutoff = int(0.04 * df['weight'].sum())
        df['weight_cumsum'] = df['weight'].cumsum()
        df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        return (df_cutoff['target'] == 1).sum() / (df['target'] == 1).sum()
    

In [10]:
train_data = np.load(OUTDIR+"train13_raw_all_data.npy")
train_labels = np.load(OUTDIR+"train13_raw_all_labels.npy")

In [9]:


test_data = np.load(OUTDIR+"train_raw_all_data.npy", mmap_mode="r+")
train_labels = np.load(OUTDIR+"train_raw_all_labels.npy")
class Data(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):        
        return torch.from_numpy(self.data[index]), index

train_dataset = Data(test_data, )
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)


In [19]:
model_name = "conv13_32_all"
model = Conv(conv_channels=32)
model_param = torch.load(OUTDIR+model_name)
model.load_state_dict(model_param)

<All keys matched successfully>

In [14]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=1/9, random_state=1, shuffle=True)
validation_data = (X_test, y_test)


In [16]:
X_test, y_test = validation_data
val_features = torch.from_numpy(X_test)
val_pred = model(val_features)
val_metrix, val_gini, val_recall = amex_metric(y_test, val_pred.detach().numpy(), return_components=True)


In [17]:
val_metrix, val_gini, val_recall

(0.8106466362523513, 0.9281171813848559, 0.6931760911198468)

In [13]:
val_metrix, val_gini, val_recall

(0.8076595984542532, 0.9279778102472538, 0.6873413866612527)

In [9]:
preds = np.load(OUTDIR+"c13_preds.npy")

In [ ]:
a, b, c = amex_metric(train_labels, preds, return_components=True)

In [13]:
preds = np.zeros(len(train_dataset))
model.eval()
for idx, (feat, indices) in enumerate(train_loader):
        batch_pred = model(feat)
        preds[indices.numpy()] = batch_pred.detach().numpy().reshape(-1, )
        #preds.append(batch_pred)
        print(amex_metric(train_labels[indices.numpy()], preds[indices.numpy()], return_components=True))

(0.9189522846823319, 0.6626991745056633, 0.7908257295939976)
(0.9117926612179975, 0.6322059953881629, 0.7719993283030802)
(0.9137001253827733, 0.630639278172394, 0.7721697017775837)
(0.910439040794739, 0.6391986391986392, 0.7748188399966891)
(0.9143557163604561, 0.6452611218568666, 0.7798084191086614)
(0.9117057372875503, 0.6510107015457788, 0.7813582194166646)
(0.907421621330389, 0.6261520737327189, 0.7667868475315539)
(0.9117512332341383, 0.6370797310278579, 0.7744154821309981)
(0.9146816478243461, 0.6332438650306749, 0.7739627564275104)
(0.911634259072624, 0.6298299845440495, 0.7707321218083367)
(0.9122821673600039, 0.6259376233714963, 0.76910989536575)
(0.9091516211022168, 0.630766223612197, 0.769958922357207)
(0.9177579459881187, 0.6546998648909477, 0.7862289054395332)
(0.9128007189670488, 0.6530612244897959, 0.7829309717284223)
(0.9095910951030811, 0.6390001908032819, 0.7742956429531815)
(0.9122655531659488, 0.6271450858034321, 0.7697053194846905)
(0.9123261570727688, 0.645030032

(0.47234747244543596, 0.09336767546683837, 0.28285757395613714)

In [25]:
%load_ext memory_profiler


In [10]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=1/9, random_state=0, shuffle=True)
validation_data = (X_test, y_test)

train_dataset = CustomerData(X_train, train_labels=y_train)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE)


In [14]:
model_name = "conv_all_13"

model = Conv(input_dim=X_train.shape[-1], conv_channels=25)
model = train_torch_model(model, train_loader, num_epochs=50, validation_data=validation_data, 
                        output_model_name=model_name)
 

0, BCE loss: 0.746, amex train: 0.033, val 0.000
0, BCE loss: 0.880, amex train: 0.146, val 0.000
0, BCE loss: 0.725, amex train: 0.220, val 0.000
0, BCE loss: 0.568, amex train: 0.288, val 0.000
0, BCE loss: 0.526, amex train: 0.314, val 0.000
0, BCE loss: 0.582, amex train: 0.345, val 0.000
0, BCE loss: 0.607, amex train: 0.355, val 0.000
0, BCE loss: 0.571, amex train: 0.358, val 0.000
0, BCE loss: 0.529, amex train: 0.384, val 0.000
0, BCE loss: 0.511, amex train: 0.386, val 0.000
0, BCE loss: 0.519, amex train: 0.391, val 0.000
0, BCE loss: 0.531, amex train: 0.378, val 0.000
0, BCE loss: 0.546, amex train: 0.387, val 0.000
0, BCE loss: 0.537, amex train: 0.388, val 0.000
0, BCE loss: 0.542, amex train: 0.388, val 0.000
0, BCE loss: 0.529, amex train: 0.396, val 0.000
0, BCE loss: 0.520, amex train: 0.399, val 0.000
0, BCE loss: 0.494, amex train: 0.387, val 0.000
1, BCE loss: 0.507, amex train: 0.408, val 0.000
1, BCE loss: 0.503, amex train: 0.387, val 0.000
1, BCE loss: 0.511, 

: 

: 

(343141, 13, 188)

# D analysis of GBM

In [5]:
import numpy as np
import pandas as pd
from pd.params import *
def get_agg_data(data_dir="train_agg_mean_q5_q95_q5_q95.npz"):
    d = np.load(OUTDIR+data_dir)
    #train_data = np.concatenate((d["d2"].astype(np.int32), d["d1"].reshape(d["d1"].shape[0], -1)), axis=1)
    train_labels = d["labels"]
    df2 = pd.DataFrame(d["d2"].astype(np.int32))
    df = pd.DataFrame(d["d1"].reshape(d["d1"].shape[0], -1))
    df = pd.concat((df2, df), axis=1,)
    df.columns = [f"c{i}" for i in range(df.shape[1])]
    cat_indices = list(np.arange(d["d2"].shape[1]))

    return df, train_labels, cat_indices

In [57]:
df, train_labels, cat_indices = get_agg_data(data_dir="train_agg_mean_q5_q95_q5_q95.npz")        

In [58]:
from sklearn.model_selection import StratifiedKFold, train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, train_labels, test_size=1/9, random_state=0, shuffle=True)
validation_data = (X_test, y_test)

In [59]:
import lightgbm as lgb 
model = lgb.Booster(model_file=OUTDIR+"lgbm_agg")


In [60]:
y_pred = model.predict(X_test)

In [61]:
amex_metric(y_test, y_pred, return_components=True)

(0.9247766447649399, 0.6687252211039951, 0.7967509329344675)

In [64]:
df = pd.DataFrame({"pred":y_pred, "target":y_test.reshape(-1)})

In [66]:
df.sort_values(by="pred", ascending=False, inplace=True)
df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
df['weight_cumsum'] = df['weight'].cumsum()
#df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        

In [16]:
four_pct_cutoff = int(0.04 * df['weight'].sum())


In [34]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)



In [18]:
df.sort_values(by=["pred", "weight"], ascending=False).reset_index(drop=True).to_csv(OUTDIR+"pred.csv")

In [36]:
df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]

In [40]:
df_cutoff[["target", "c0"]].value_counts()

target  c0
1.0     13    7440
0.0     13     932
1.0     12     189
        10     141
        8      126
        9      110
        2      109
        7      105
        11     102
        6      101
        3      101
        5       94
        1       82
        4       71
0.0     9       25
        12      22
        1       18
        4       17
        3       17
        2       14
        10      14
        7       10
        6       10
        11       9
        8        8
        5        6
dtype: int64

In [24]:
a = df.loc[df['weight_cumsum'] >= four_pct_cutoff]
a[a.target == 1].shape

(4345, 4)

In [42]:
a = df_cutoff[df_cutoff.c0==13]

In [ ]:
import plotly.express as px
fig = px.line(a, x="feature", y=['gbm_recall', 'gbm_gini'], hue="target")
fig.show()

# Error Analysis of the C13 

In [49]:
train_data = np.load(OUTDIR+"train13_raw_all_data.npy")
train_labels = np.load(OUTDIR+"train13_raw_all_labels.npy")

In [50]:
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=1/9, random_state=0, shuffle=True)
validation_data = (X_test, y_test)

In [53]:
import joblib
import lightgbm as lgb 

model = joblib.load(OUTDIR+f'lgbm13.pkl')

In [56]:
y_pred = model.predict(X_test.reshape(X_test.shape[0], -1))
amex_metric(y_test, y_pred, return_components=True)

(0.9323274209347154, 0.6966805400466958, 0.8145039804907056)

In [67]:
y_pred.shape 

(50991,)

In [68]:
import json 
with open(OUTDIR+'train_agg1_mean_q5_q95_q5_q95_id.json', 'r') as f:
            train_id_dict = json.load(f)

In [83]:
train_customers_count = pd.read_parquet(TRAINDATA).customer_ID.value_counts().to_dict()


In [ ]:
train_id_dict1 = {val:key for key, val in train_id_dict.items()}

# EA for Ensemble
 - get the indices of the c13 and other 
 - split data into the train- val for the 13 and the rest 
 - investigate how use other probs in combination with the C13

In [13]:
from pd.utils import get_customers_data_indices, get_customers_id_from_indices


In [9]:
df = pd.read_csv(PREDDIR+"train_pred.csv", index_col=0)

In [14]:
c13_indices, other_indices = get_customers_data_indices(num_data_points=[13], id_dir=f'train_agg{1}_mean_q5_q95_q5_q95_id.json')
c13 = get_customers_id_from_indices(c13_indices)
co = get_customers_id_from_indices(other_indices)

In [69]:
for col in df.columns[1:]:
    print(col, amex_metric(df.target.values.reshape(-1), df[col].values.reshape(-1), return_components=True))

lgbm13_agg1 (0.7838867741175084, 0.9155198706506092, 0.6522536775844077)
lgbm_agg1 (0.7932525928055224, 0.923782594975841, 0.6627225906352038)
xgbm13_agg1 (0.7803792021838198, 0.9141599595566524, 0.6465984448109873)
xgbm_agg1 (0.7890485361667914, 0.9213410888953358, 0.6567559834382469)
catb13_agg1 (0.782928953236126, 0.9160363156014134, 0.6498215908708386)
xgbm13_agg2 (0.019606480060928782, -0.00039050034200617577, 0.03960346046386374)
catb_agg1 (0.7912651133544574, 0.9222397227872801, 0.6602905039216347)
xgbm_agg2 (0.019552792231331984, -0.0004389673264766379, 0.039544551789140604)
xgbm13_agg3 (0.01948767129421065, -0.0004766384261543881, 0.03945198101457569)
xgbm13_agg0 (0.019461779178026235, -0.00040218978411651175, 0.03932574814016898)
catb13_agg0 (0.019988080942495445, -3.965930193474984e-05, 0.04001582118692564)
catb13_agg3 (0.020047814758487503, 0.00016396357965383395, 0.03993166593732117)
xgbm_agg3 (0.019804481002760232, -0.00023013315723578804, 0.03983909516275625)
xgbm_agg0 (

In [21]:

amex_metric(df.loc[c13].target.values.reshape(-1), df.loc[c13].xgbm13_agg1.values.reshape(-1), return_components=True)

(0.8142794254216831, 0.9309572057462473, 0.6976016450971189)

In [23]:
amex_metric(df.loc[co].target.values.reshape(-1), df.loc[co].xgbm_agg1.values.reshape(-1), return_components=True)

(0.6620866932343902, 0.8499996215624771, 0.47417376490630325)

AttributeError: 'DataFrame' object has no attribute 'lgbm_agg0'

In [33]:
amex_metric(df.target.values.reshape(-1), df.xgbm_agg1.values.reshape(-1), return_components=True)

(0.7890485361667914, 0.9213410888953358, 0.6567559834382469)

In [34]:
# Take the c13 from 13 preds and the rest from others

In [ ]:
amex_metric(df.loc[c13].target.values.reshape(-1), df.loc[c13].xgbm13_agg1.values.reshape(-1), return_components=True)

In [63]:
df["mp"] = 0
df["mp"].loc[c13]= df.loc[c13].lgbm13_agg1.values.reshape(-1)
df["mp"].loc[co] = df.loc[co].lgbm_agg1.values.reshape(-1)

/var/folders/t2/_nykd9tn43dfx142nv5r8l900000gn/T/ipykernel_20168/2280686262.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["mp"].loc[c13]= df.loc[c13].lgbm13_agg1.values.reshape(-1)


In [64]:
amex_metric(df.target.values.reshape(-1), df.mp.values.reshape(-1), return_components=True)

(0.7931060130914598, 0.9236156684221223, 0.6625963577607972)

In [52]:
df

,target,lgbm13_agg1,lgbm_agg1,xgbm13_agg1,xgbm_agg1,catb13_agg1,xgbm13_agg2,catb_agg1,xgbm_agg2,xgbm13_agg3,...,catb13_agg4,catb_agg4,xgbm13_agg4,conv13_agg1,lgbm13_agg4,lgbm_agg4,xgbm_p1_agg4,xgbm_p1_agg0,xgbm13_p1_agg2,mp
customer_ID,,,,,,,,,,,,,,,,,,,,,
0000099d6bd597052cdcda90ffabf56573fe9d7c79be5fbac11a8ed792feb62a,0,0.000785,0.000865,0.000929,0.001096,0.000507,0.084060,0.000377,0.411908,0.086643,...,0.319579,0.233948,0.248878,0.057720,0.223292,0.221504,0.278566,0.473031,0.394507,0.000929
00000fd6641609c6ece5454664794f0340ad84dddce9a267a310b5ae68e9d8e5,0,0.001388,0.001169,0.001767,0.001736,0.001521,0.001192,0.001198,0.001043,0.001298,...,0.007746,0.009628,0.030730,0.005716,0.004263,0.007139,0.010354,0.001710,0.008161,0.001767
00001b22f846c82c51f6e3958ccd81970162bae8b007e80662ef27519fcc18c1,0,0.000769,0.000928,0.001328,0.001397,0.001477,0.708267,0.001522,0.682479,0.721939,...,0.926558,0.852145,0.845415,0.989080,0.898360,0.875647,0.923688,0.758849,0.747954,0.001328
000041bdba6ecadd89a52d11886e8eaaec9325906c9723355abb5ca523658edc,0,0.006624,0.006822,0.014978,0.015899,0.005526,0.004889,0.007509,0.005005,0.004198,...,0.002244,0.002142,0.228898,0.036013,0.002851,0.001793,0.002868,0.006334,0.011321,0.014978
00007889e4fcd2614b6cbe7f8f3d2e5c728eca32d9eb8ad51ca8b8c4a24cefed,0,0.001455,0.001415,0.002276,0.001929,0.003260,0.173564,0.002790,0.097903,0.141820,...,0.686741,0.532207,0.869929,0.012256,0.613125,0.569500,0.554354,0.130969,0.166047,0.002276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffff41c8a52833b56430603969b9ca48d208e7c192c6a4081a6acc28cf4f8af7,0,0.005207,0.005179,0.003260,0.002894,0.003879,0.080117,0.005118,0.087961,0.070516,...,0.783628,0.803240,0.772201,0.968079,0.735141,0.772904,0.748484,0.097006,0.118654,0.003260
ffff518bb2075e4816ee3fe9f3b152c57fc0e6f01bf7fdd3e5b57cfcbee30286,0,0.012799,0.013453,0.020431,0.015333,0.023504,0.641878,0.021071,0.703336,0.671222,...,0.007948,0.007321,0.289117,0.025283,0.010784,0.009717,0.010460,0.641897,0.664796,0.020431
ffff9984b999fccb2b6127635ed0736dda94e544e67e026eee4d20f680639ff6,0,0.001078,0.001216,0.002635,0.002300,0.001189,0.002700,0.001184,0.003638,0.003095,...,0.050234,0.067844,0.086980,0.003368,0.037413,0.057313,0.043447,0.006240,0.039908,0.002635


In [ ]:
amex_metric(train_labels[indices.numpy()], preds[indices.numpy()], return_components=True)

In [ ]:
df.sort_values(by="pred", ascending=False, inplace=True)
df['weight'] = df['target'].apply(lambda x: 20 if x==0 else 1)
df['weight_cumsum'] = df['weight'].cumsum()
#df_cutoff = df.loc[df['weight_cumsum'] <= four_pct_cutoff]
        